# Intro to 3D CNNs
A description and demo notebook to go through creating a 3D CNN and using it with dummy data

## 1. Setting up dummy data

Before learning how to use the cores, let's create a dummy data images. This data will be similar to a batch of images.

Throughout the notebook we will refer to the elements of this shape in the following manner:

[1] is the number of channels (can be input, hidden, output)

[144] is the height of image or feature maps

[256] is the height of image or feature maps

[32] is the batch size, which is not as relevant for understanding the material in this notebook.

In [1]:
# To access to neuropixel_predictor
import sys
sys.path.append('../')

# Essential Imports
import torch


images = torch.ones(32, 1, 144, 256)


## 2. Using Stacked 3D Core

In [2]:
stacked3dcore_config = {
    # core args
    'input_channels': 1,
    'input_kernel': 9,
    'hidden_kernel': 7,
    'hidden_channels': 64,
    'layers': 4
}

In [3]:
from neuropixel_predictor.layers.cores import Basic3dCore 

stacked3d_core = Basic3dCore(**stacked3dcore_config)
stacked3dcore_out = stacked3d_core(images)
print(stacked3dcore_out.shape)

/Users/tarek/Documents/Repos/neuropixel-predictor/notebooks/../neuropixel_predictor/layers/cores/base.py:82: UserWarning: The batch_norm is applied to all layers
  warnings.warn(f"The {attr} is applied to all layers", UserWarning)
/Users/tarek/Documents/Repos/neuropixel-predictor/notebooks/../neuropixel_predictor/layers/cores/base.py:82: UserWarning: The hidden_channels is applied to all layers
  warnings.warn(f"The {attr} is applied to all layers", UserWarning)
/Users/tarek/Documents/Repos/neuropixel-predictor/notebooks/../neuropixel_predictor/layers/cores/base.py:82: UserWarning: The bias is applied to all layers
  warnings.warn(f"The {attr} is applied to all layers", UserWarning)
/Users/tarek/Documents/Repos/neuropixel-predictor/notebooks/../neuropixel_predictor/layers/cores/base.py:82: UserWarning: The batch_norm_scale is applied to all layers
  warnings.warn(f"The {attr} is applied to all layers", UserWarning)


TypeError: unsupported operand type(s) for %: 'tuple' and 'int'